In [2]:
import numpy as np
import warnings
import cv2

## Preparation

In [3]:
grainger_image = cv2.imread('grainger.jpg').astype(np.uint8)

## Naive Method: Increase blue channel

In [4]:
blue_tint = np.array([0.5, 0.2, 0.1])
# Add a blue tint ot he image
blue_img = grainger_image * blue_tint

# Normalize image
blue_img_n = (255 * blue_img/blue_img.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (blue_img_n/255) ** gamma, dtype="uint8")
cv2.imwrite("night_final.png", gamma_img)

True

## Intermediate Method

In [5]:
def turn_day_to_night(image, alpha):
    image_width, image_height, color_channels = image.shape
    # Convert image to XYZ color space
    xyz_image = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)

    # For the Scotopic Luminance of the image
    x = xyz_image[:,:,0]
    y = xyz_image[:,:,1]
    z = xyz_image[:,:,2]
    scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)
    scotopic_luminance = np.clip(np.nan_to_num(scotopic_luminance), 0, 255).astype(np.uint8)

    # Create a scotopic luminance image with 3 color channels
    scotopic_lum_img = np.zeros((image_width, image_height, color_channels)).astype(np.uint8)
    blue = np.array([.9, .5, .5]).astype(np.float64)
    k = 1
    for c in range(color_channels):
        scotopic_lum_img[:,:,c] = scotopic_luminance * blue[c] * k

    # Blend original and scotopic images pixel by pixel
    night_image = image * (1 - alpha) + scotopic_lum_img * alpha

    # Adjust contrast
    night_image = cv2.convertScaleAbs(night_image, alpha=0.8, beta=0)

    # Adding Gaussian blur to image
    kernel_size = 5
    night_image = cv2.GaussianBlur(night_image, (kernel_size, kernel_size), 2)

    #Adding noise to image
    noise_filter = np.zeros((image_width, image_height, color_channels), np.uint8)
    cv2.randu(noise_filter,(0),(10))
    
    result_image = night_image + noise_filter
    return result_image

In [6]:
warnings.filterwarnings('ignore')

alpha = 0.7
night_image = turn_day_to_night(grainger_image, alpha)
cv2.imwrite("night_final.png", night_image)

True

## Advanced Method: Remove illumination, decrease exposure, relight with night illuminants, and add noise

In [7]:
''' We begin the Advanced Method by taking our source image as a demosaiced day image'''
I_day = grainger_image

Normalized image:
$$ \mathbf{I}_n = (\mathbf{I}_{day}-b_l) / (w_l-b_l) $$

In [8]:
# The white level is the brightest pixel value in the image; The black level is the darkest pixel in the image
wl = np.max(np.sum(I_day, axis=2))
bl = np.min(np.sum(I_day, axis=2))

I_n = (I_day - bl) / (wl - bl)

cv2.imwrite("I_n.png", I_n * 255.0)

True

White-balanced image:
$$ \mathbf{I}_w = \mathbf{I}_n\mathbf{L}_{day} $$
where
$$ {L}_{day} = diag(\frac{1}{b}, \frac{1}{g}, \frac{1}{g}, \frac{1}{r}) $$

In [9]:
# As our image is a numpy array, we normalize the image using numpy functions for efficiency

# https://chat.openai.com/share/f90475c2-2156-42c5-afba-2b8a22fee9f1
L_day = np.identity(I_n.shape[-1])
I_w = I_n @ L_day

I_w = np.clip((I_n*1.0 / I_n.mean(axis=(0,1))), 0, 1)
cv2.imwrite("I_w.png", I_w * 255)

True

### Lowering brightness

`D`: dictionary for the normalized mean intensity value of each Bayer image
(could just make list with index as key for the image)

In [10]:
nighttime_images = np.array([I_day])

# contains normalized mean intensity values of each Bayer image
D = np.array([np.mean(img) / np.max(img) for img in nighttime_images])

# Hardcoded global scale factor for demonstration purposes
global_scale_factor_d = 0.3 #np.random.choice(D) 
print(global_scale_factor_d)

I_e = I_w * global_scale_factor_d
cv2.imwrite("I_e.png", I_e * 255)


0.3


True

### Adding illuminants

`L`: dictionary for nighttime illuminants

In [11]:
# We first define a function for creating gaussian filters

# Made with help from chatgpt
def generate_2d_gaussian_array(shape, sigma=1.0, center=None):
    """
    Generate a 2-D Gaussian array.

    Parameters:
        shape (tuple): Shape of the output array.
        sigma (float): Standard deviation of the Gaussian filter.
        center (tuple): Center of the Gaussian filter. Default is None, which sets the center to the center of the array.

    Returns:
        numpy.ndarray: 2-D Gaussian array with the specified shape.
    """
    if center is None:
        center = (shape[0] // 2, shape[1] // 2)

    x, y = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]))
    x -= center[0]
    y -= center[1]

    exponent = -(x**2 + y**2) / (2 * sigma**2)
    gaussian_array = np.exp(exponent) / (2 * np.pi * sigma**2)

    return gaussian_array

In [12]:
i_r = np.zeros(I_e.shape)

# Define the nighttime illuminants
night_time_illuminant = [[.5, .3, .3], [0, 1, 1], [0, 1, 1]]

# Add masked illuminants to result image
M = generate_2d_gaussian_array((1800,1200),sigma=(500))

i_r[:,:,0] += (I_e * night_time_illuminant[0])[:,:,0] * 2 
i_r[:,:,1] += (I_e * night_time_illuminant[0])[:,:,1] * 2
i_r[:,:,2] += (I_e * night_time_illuminant[0])[:,:,2] * 2


M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(200,600))
i_r[:,:,0] += (I_e * night_time_illuminant[1])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[1])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[1])[:,:,2] * M * 3e4

M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(1600,600))
i_r[:,:,0] += (I_e * night_time_illuminant[2])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[2])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[2])[:,:,2] * M * 3e4

M = generate_2d_gaussian_array((1800,1200),sigma=(150), center=(900,600))
i_r[:,:,0] += (I_e * night_time_illuminant[2])[:,:,0]* M * 3e4
i_r[:,:,1] += (I_e * night_time_illuminant[2])[:,:,1] * M * 3e4
i_r[:,:,2] += (I_e * night_time_illuminant[2])[:,:,2] * M * 3e4


cv2.imwrite("I_r.png", (i_r * 255))

True

In [13]:
i_night = i_r * (wl - bl) + bl
cv2.imwrite("I_night.png", i_night)

True

### Adding noise

In [14]:
noise = np.random.normal(0,5, i_r.shape)
cv2.imwrite("I_noise.png", (i_night * 255) + noise)

True